# Creating confidence intervals in python

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import t
pd.set_option('display.max_columns', 30) # set maximun 30 columns to be seen

Your goal is to analyse data which is the result of a study that examined
differences in a number of sleep variables between napping and non-napping toddlers. Some of these
sleep variables included: Bedtime (lights-off time in decimalized time), Night Sleep Onset Time (in
decimalized time), Wake Time (sleep end time in decimalized time), Night Sleep Duration (interval
between sleep onset and sleep end in minutes), and Total 24-Hour Sleep Duration (in minutes). Note:
[Decimalized time](https://en.wikipedia.org/wiki/Decimal_time) is the representation of the time of day using units which are decimally related.   


The 20 study participants were healthy, normally developing toddlers with no sleep or behavioral
problems. These children were categorized as napping or non-napping based upon parental report of
children’s habitual sleep patterns. Researchers then verified napping status with data from actigraphy (a
non-invasive method of monitoring human rest/activity cycles by wearing of a sensor on the wrist) and
sleep diaries during the 5 days before the study assessments were made.


You are specifically interested in the results for the Bedtime, Night Sleep Duration, and Total 24-
Hour Sleep Duration. 

Reference: Akacem LD, Simpkin CT, Carskadon MA, Wright KP Jr, Jenni OG, Achermann P, et al. (2015) The Timing of the Circadian Clock and Sleep Differ between Napping and Non-Napping Toddlers. PLoS ONE 10(4): e0125181. https://doi.org/10.1371/journal.pone.0125181

In [2]:
df = pd.read_csv("nap_no_nap.csv")

In [3]:
df

,id,sex,age (months),dlmo time,days napped,napping,nap lights outl time,nap sleep onset,nap midsleep,nap sleep offset,nap wake time,nap duration,nap time in bed,night bedtime,night sleep onset,sleep onset latency,night midsleep time,night wake time,night sleep duration,night time in bed,24 h sleep duration,bedtime phase difference,sleep onset phase difference,midsleep phase difference,wake time phase difference
0,1,female,33.7,19.24,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.45,20.68,0.23,1.92,7.17,629.40,643.00,629.40,-1.21,-1.44,6.68,11.93
1,2,female,31.5,18.27,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.23,19.48,0.25,1.09,6.69,672.40,700.40,672.40,-0.96,-1.21,6.82,12.42
2,3,male,31.9,19.14,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.60,20.05,0.45,1.29,6.53,628.80,682.60,628.80,-0.46,-0.91,6.15,11.39
3,4,female,31.6,19.69,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.46,19.50,0.05,1.89,8.28,766.60,784.00,766.60,0.23,0.19,6.20,12.59
4,5,female,33.0,19.52,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.21,19.65,0.45,1.30,6.95,678.00,718.00,678.00,0.31,-0.13,5.78,11.43
5,6,female,36.2,18.22,4,1,14.00,14.22,15.00,15.78,16.28,93.75,137.00,19.95,20.25,0.29,1.26,6.28,602.20,653.80,695.95,-1.73,-2.03,7.05,12.06
6,7,male,36.3,19.28,1,1,14.75,15.03,15.92,16.80,16.08,106.00,80.00,20.60,20.96,0.36,2.12,7.27,618.40,655.40,724.40,-1.32,-1.68,6.84,11.99
7,8,male,30.0,21.06,5,1,13.09,13.43,14.44,15.46,15.82,121.60,163.80,22.01,22.53,0.51,2.92,7.31,526.80,582.40,648.40,-0.95,-1.47,5.86,10.25
8,9,male,33.2,19.38,2,1,14.41,14.42,15.71,17.01,16.60,155.50,131.25,20.24,20.37,0.13,1.60,6.82,626.80,660.33,782.30,-0.86,-0.99,6.22,11.44
9,10,female,37.1,19.93,3,1,13.12,13.42,14.31,15.19,15.30,106.67,130.67,20.78,21.63,0.84,2.20,6.52,549.50,626.00,656.17,-0.76,-1.82,6.21,10.59


__Question__ : What variable is used in the column "napping" to indicate a toddler takes a nap?

In [4]:
df.napping.unique()

array([0, 1])

__Answer__ : Variable "1" is used.

__Question__ : What is the sample size $n$? What is the sample size for toddlers who nap, $n_1$, and toddlers who don't nap, $n_2$?

In [5]:
n = df.shape[0]
n_1 = df.query("napping == 1").shape[0]
n_2 = df.query("napping == 0").shape[0]

print(f"Answer: Sample size n is {n}, nap_toddlers n_1 is {n_1} an no_nap_toddlers n_2 is {n_2}.")

Answer: Sample size n is 20, nap_toddlers n_1 is 15 an no_nap_toddlers n_2 is 5.


### Average bedtime confidence interval for napping and non napping toddlers
Create two 95% confidence intervals for the average bedtime, one for toddler who nap and one for toddlers who don't.

Before any analysis, we will convert 'night bedtime' into decimalized time. Using following code
```python
df.loc[:,'night bedtime'] = np.floor(df['night bedtime'])*60 + np.round(df['night bedtime']%1,2 )*100
```

In [6]:
np.floor(df['night bedtime'])*60

0     1200.0
1     1140.0
2     1140.0
3     1140.0
4     1140.0
5     1140.0
6     1200.0
7     1320.0
8     1200.0
9     1200.0
10    1140.0
11    1200.0
12    1200.0
13    1200.0
14    1200.0
15    1200.0
16    1200.0
17    1200.0
18    1140.0
19    1200.0
Name: night bedtime, dtype: float64

In [7]:
np.round(df['night bedtime']%1, 2)*100

0     45.0
1     23.0
2     60.0
3     46.0
4     21.0
5     95.0
6     60.0
7      1.0
8     24.0
9     78.0
10    45.0
11    18.0
12    22.0
13    26.0
14    28.0
15    46.0
16    43.0
17     2.0
18    50.0
19    18.0
Name: night bedtime, dtype: float64

In [8]:
df.loc[:,'night bedtime'] = np.floor(df['night bedtime'])*60 + np.round(df['night bedtime']%1,2 )*100
df['night bedtime']

0     1245.0
1     1163.0
2     1200.0
3     1186.0
4     1161.0
5     1235.0
6     1260.0
7     1321.0
8     1224.0
9     1278.0
10    1185.0
11    1218.0
12    1222.0
13    1226.0
14    1228.0
15    1246.0
16    1243.0
17    1202.0
18    1190.0
19    1218.0
Name: night bedtime, dtype: float64

Now, isolate the column 'night bedtime' for those who nap into a new variable, and those who didn't nap into another new variable. 

In [9]:
bedtime_nap = df.query("napping == 1")["night bedtime"]
bedtime_no_nap = df.query("napping == 0")["night bedtime"]

Now find the sample mean bedtime for nap and no_nap.

In [30]:
nap_mean_bedtime = np.mean(bedtime_nap)
no_nap_mean_bedtime = np.mean(bedtime_no_nap)

Now find the standard error for $\bar{X}_{nap}$ and $\bar{X}_{no\ nap}$.

In [31]:
x_nap = np.std(bedtime_nap, ddof=1) / np.sqrt(n_1)
x_no_nap = np.std(bedtime_no_nap, ddof=1) / np.sqrt(n_2)

**Question**: Given our sample sizes of $n_1$ and $n_2$ for napping and non napping toddlers respectively, how many degrees of freedom ($df$) are there for the associated $t$ distributions?

Answer: Degree of freedom ($df_1$) for $n_1$ = 14 and degree of freedom ($df_2$) for $n_2$ = 4.

To build a 95% confidence interval, what is the value of t\*?  You can find this value using the percent point function: 
```
from scipy.stats import t

t.ppf(probabiliy, df)
```
This will return the quantile value such that to the left of this value, the tail probabiliy is equal to the input probabiliy (for the specified degrees of freedom). 

Example: to find the $t^*$ for a 90% confidence interval, we want $t^*$ such that 90% of the density of the $t$ distribution lies between $-t^*$ and $t^*$.

Or in other words if $X \sim t(df)$:

P($-t^*$ < X < $t^*$) = .90

Which, because the $t$ distribution is symmetric, is equivalent to finding $t^*$ such that:  

P(X < $t^*$) = .95

So the $t^*$ for a 90% confidence interval, and lets say df=10, will be:

t_star = t.ppf(.95, df=10)


In [36]:
# Find the t_stars for the 95% confidence intervals
nap_t_star = t.ppf(q=0.975, df=n_1-1)
no_nap_t_star = t.ppf(q=0.975, df=n_2-1)

**Quesion**: What is $t^*$ for nap and no nap?

In [37]:
print(f"Answer: nap_t* is {nap_t_star} and no nap_t* is {no_nap_t_star}.")

Answer: nap_t* is 2.1447866879169273 and no nap_t* is 2.7764451051977987.


Now to create our confidence intervals. For the average bedtime for nap and no nap, find the upper and lower bounds for the respective confidence intervals.

In [34]:
def confidence_interval(x, mean, t_star):
    ci_lower = round(mean - t_star * x, 2)
    ci_upper = round(mean + t_star * x, 2)
    return {"lower": ci_lower, "upper": ci_upper}

In [35]:
# upper/lower bounds for "nap"
nap_ci = confidence_interval(x_nap, nap_mean_bedtime , nap_t_star)

# upper/lower bounds for "no_nap"
no_nap_ci = confidence_interval(x_no_nap, no_nap_mean_bedtime, no_nap_t_star)

print(f"nap --> lower bound: {nap_ci['lower']} | upper bound: {nap_ci['upper']}")
print(f"no_nap --> lower bound: {no_nap_ci['lower']} | upper bound: {no_nap_ci['upper']}")

nap --> lower bound: 1217.4 | upper bound: 1248.73
no_nap --> lower bound: 1158.3 | upper bound: 1223.7


**Question**: What are the 95% confidence intervals, rounded to the nearest ten, for the average bedtime (in decimalized time) for toddlers who nap and for toddlers who don't nap? 

CI = $\bar{X} \pm \ t^* \cdot s.e.(\bar{X})$